In [17]:
import pandas as pd
import numpy as np

In [18]:
names_df = pd.read_csv("raw_data/Names.csv",usecols = [0,1])
address_df = pd.read_csv("raw_data/address.csv").sample(100, random_state = 2022, ignore_index = True)
cardno_df = pd.read_csv("raw_data/cardno.csv",usecols = [0]).sample(100, random_state = 2022, ignore_index = True)
cardno_df

,Card_Number
0,2357-9896-6403-5970
1,6514-4444-5642-2615
2,4783-2852-8264-4966
3,4856-2191-9879-6388
4,4138-5166-7490-1188
...,...
95,7837-4036-5999-1672
96,3652-5528-4808-8216
97,7658-7645-4568-1461
98,4093-1634-8086-4601


### Customers data

**Names attribute**

In [19]:
customer_df = names_df.iloc[:100,:].copy()
customer_df.rename(columns = {'First Name': "fname", "Last Name": "lname"}, inplace = True)
customer_df

,fname,lname
0,Mechelle,Stoneman
1,Tyesha,Freitag
2,Dean,Stoecker
3,Annelle,Pickney
4,Margareta,Tacy
...,...,...
95,Glenda,Woolfolk
96,Alyson,Hosack
97,Francis,Sines
98,Riley,Bagnall


**id, age, email, address**

In [20]:
np.random.seed(2022)
email_suffix = ['@yahoo.com', '@gmail.com', '@hotmail.com', '@nyu.edu']
email_suffix_list = np.random.choice(email_suffix,100)

age = np.random.randint(18, 60, 100)
# print(age)

cid = np.random.randint(111111, 999999, 100)
# print(cid)

email_list = [f.lower()+l.lower()+e for f,l,e in zip(customer_df['fname'], customer_df['lname'], email_suffix_list)]
# print(email_list)

address_list = [', '.join(val) for val in address_df.astype(str).values.tolist()]
# print(address_list)


customer_df['customer_id'] = cid
customer_df['age'] = age
customer_df['email'] = email_list
customer_df['address'] = address_list

In [21]:
customer_df = customer_df[['customer_id', 'fname', 'lname', 'age', 'email', 'address']]

In [216]:
customer_df.to_csv('final_data/Customers.csv', index = False)

## Cards data

In [213]:
np.random.seed(2022)
customer_sample = customer_df.sample(100, replace = True, random_state = 2022)

expiry_month = np.random.randint(1,13, 100)
# print(expiry_month)

expiry_year = np.random.randint(2024,2030, 100)
# print(expiry_year)


cards_df = pd.DataFrame()
cards_df['card_number'] = [''.join(val.strip().split('-')) for val in cardno_df['Card_Number']]
cards_df['customer_id'] = list(customer_sample['customer_id'])
cards_df['billing_address'] = list(customer_sample['address'])
cards_df['expiry_month'] = expiry_month
cards_df['expiry_year'] = expiry_year


In [214]:
cards_df.to_csv('final_data/HasCards.csv', index = False)

## Sold by

In [3]:
product_df = pd.read_csv('final_data/products.csv')
seller_df = pd.read_csv('final_data/sellers.csv')

In [16]:
np.random.seed(2022)
seller_id = np.random.choice(seller_df['seller_id'],len(product_df['product_id']))
seller_id

soldby_df = pd.DataFrame({'seller_id':seller_id, 'product_id':product_df['product_id']})
soldby_df.to_csv('final_data/SoldBy.csv', index = False)

## Bought in

In [48]:
# import uuid
# uuids = [uuid.uuid4().hex for i in range(1000)]
# uuid_df = pd.DataFrame({'uuid':uuids})
# uuid_df.to_csv('final_data/uuid.csv', index = False)

In [92]:
uuid_df = pd.read_csv('final_data/uuid.csv')
product_df = pd.read_csv('final_data/products.csv')

In [114]:
np.random.seed(2022)
n_order = 40
gen_order_id = list(uuid_df['uuid'][:n_order].values)
n_items = np.random.randint(1, 11, size = n_order)

order_id = []
product_id = []
quantity = []

for _id, _ni in zip(gen_order_id, n_items):
    t_products = np.random.choice(product_df['product_id'], _ni, replace = False)
    t_quantity = np.random.randint(1, 50, size = _ni)
    for i in range(_ni):    
        order_id.append(_id)
        product_id.append(t_products[i])
        quantity.append(t_quantity[i])
        
boughtin_df = pd.DataFrame({'order_id':order_id, 'product_id':product_id, 'quantity':quantity})
boughtin_df.to_csv('final_data/boughtin.csv', index = False)

## Shipped from

In [116]:
uuid_df = pd.read_csv('final_data/uuid.csv')

In [154]:
np.random.seed(2022)
n_order = 40
gen_order_id = list(uuid_df['uuid'][:n_order].values)


warehouse = [{'name':'C-bay Warehouse1', 'location': '3412-3416 Garfield Avenue Commerce, CA 90040', 'zipcode':90040}, 
             {'name':'C-bay Warehouse2', 'location': '135 West 36th Street, New York, NY, 10018', 'zipcode':10018}]

all_warehouses = np.random.choice(warehouse, size = len(gen_order_id))


ware_name = [ent['name'] for ent in all_warehouses]
ware_loc = [ent['location'] for ent in all_warehouses]
ware_zip = [ent['zipcode'] for ent in all_warehouses]

shippedfrom_df = pd.DataFrame({'order_id':gen_order_id, 'name':ware_name, 'location': ware_loc, 'zipcode':ware_zip})
shippedfrom_df.to_csv('final_data/ShippedFromWarehouse.csv', index = False)

## ordered by

Create table OrderedByappliedon(
	order_id varchar(16) primary key,
    customer_id varchar(20) not null,
	order_placed_on date not null,
	total_amount real not null,
	tax_amount real not null,
	delivery_charges real not null,
	billed_amount real not null,
	coupon_id varchar(16),
	foreign key(coupon_id) references  Coupons(Coupon_id)
	Foreign key (customer_id) references Customers(customer_id)
);


In [226]:
uuid_df = pd.read_csv('final_data/uuid.csv')
customer_df = pd.read_csv('final_data/Customers.csv')
coupon_df = pd.read_csv('final_data/coupons.csv')

In [283]:
np.random.seed(2022)
from faker import Faker
fake = Faker()
Faker.seed(2022)

n_order = 40
gen_order_id = list(uuid_df['uuid'][:n_order].values)

gen_customer_id = np.random.choice(customer_df['customer_id'], size = n_order)
gen_order_date = [fake.date_between(start_date='-2y', end_date='now').strftime('%m-%d-%Y') for i in range(40)]

order_coupon = np.random.choice(coupon_df['coupon_id'], size = n_order)

ORDER = uuid_df[:40]
or_bo = pd.merge(ORDER, boughtin_df, left_on = 'uuid', right_on = 'order_id')
or_bo_pro = pd.merge(or_bo, product_df, left_on = 'product_id', right_on = 'product_id')
or_bo_pro_df = or_bo_pro[['order_id','product_id', 'quantity', 'price']].copy()
or_bo_pro_df['total_amount'] = or_bo_pro_df['quantity'] * or_bo_pro_df['price']
order_total = or_bo_pro_df[['order_id','total_amount']].groupby('order_id').sum().reset_index()

order_total['customer_id'] = gen_customer_id
order_total['order_placed_on'] = gen_order_date
order_total['tax_amount'] = order_total['total_amount'] * 0.18
order_total['delivery_charges'] = order_total['total_amount'] * 0.05

billed_ammount = []
rand_coupon = []
for c, t in zip(order_coupon, (order_total['total_amount'] + order_total['tax_amount'] + order_total['delivery_charges'])):
    if np.random.choice([True, False]):
        rand_coupon.append(c)
        dis = coupon_df[coupon_df['coupon_id'] == c]['discount'].values[0]
        billed = t - t*dis
        billed_ammount.append(billed)
        
    else:
        rand_coupon.append('')
        billed_ammount.append(t)
    

order_total['coupon_id'] = rand_coupon
order_total['billed_amount'] = billed_ammount

orderedby_df = order_total[['order_id', 'customer_id', 'order_placed_on', 'total_amount', 'tax_amount','delivery_charges', 'billed_amount', 'coupon_id']]

orderedby_df.to_csv('final_data/OrderedByappliedon.csv', index = False)

In [275]:

coupon_df[coupon_df['coupon_id'] == 'WAFANK']['discount'].values[0]

0.08

In [276]:
3542 - 3542*0.08

3258.64